# Corona Health - Physiological Health of adults (`Heart`) Study

## Purpose of this Notebook
- [ ] Clean the dataset and save cleaned version
- [x] Get an statistical overview
    - [x] How many users?
    - [x] How many assessments (= filled out questionnaires)?
    - [x] Date range of the dataset?
    - [x] User-assessment distribution
- [ ] Potential target for classification?
- [ ] Potential features for classification?
    
    

#### Statistical overview

In [20]:
# imports
import pandas as pd
from datetime import date, datetime

In [21]:
# to import own modules, a workaround - assuming the curent working directory is the folder (your/local/path/UsAs/src/d01_analysis
import sys
sys.path.insert(0, "../..")

from src.d00_utils import helpers

In [22]:
# read in dataframe
# assuming current working directory is the folder (your/local/path/UsAs/src/d01_analysis)
df = pd.read_csv('../../data/d01_raw/ch/22-07-01_rki_heart_followup.csv')

In [23]:
# create result dict to save to disk
result = dict()

In [24]:
# Number of unique users
print('Number of user:\t', df.user_id.nunique())
result['n_users'] = df.user_id.nunique()

Number of user:	 929


In [25]:
# number of unique answers
print('Number of assessments:\t', df.answer_id.nunique())
result['n_assessments'] = df.answer_id.nunique()

Number of assessments:	 5006


In [26]:
# date range
form = '%Y-%m-%d %H:%M:%S'
date_start = df.created_at.min()
result['First assessment from'] = date_start
date_start = datetime.strptime(date_start, form)
date_end = df.created_at.max()
result['Latest assessment from'] = date_end
date_end = datetime.strptime(date_end, form)



print('Start:\t', date_start)
print('End:\t', date_end)

delta = date_end.date()-date_start.date()

print('\nDate Range in')
print('Years:\t', delta.days/365)
print('Months:\t', delta.days/12)
print('Days:\t', delta.days)


result['Time range in days'] = delta.days

Start:	 2020-07-28 13:55:53
End:	 2022-02-03 21:16:26

Date Range in
Years:	 1.5205479452054795
Months:	 46.25
Days:	 555


#### Distribution of filled out questionnaires

In [27]:
bins = [0, 1, 2, 3, 5, 10, 100, 1000]
ser = pd.cut(df.user_id.value_counts(), bins = bins).value_counts().sort_index()
ser
result.update(dict(ser))

### Average period length between two filled out questionnaires

In [28]:
# res = result
res = helpers.find_mode_period_length(df, form='%Y-%m-%d %H:%M:%S', date_col_name='created_at')
result.update(res)

### Statistical overview

In [29]:
result

{'n_users': 929,
 'n_assessments': 5006,
 'First assessment from': '2020-07-28 13:55:53',
 'Latest assessment from': '2022-02-03 21:16:26',
 'Time range in days': 555,
 Interval(0, 1, closed='right'): 311,
 Interval(1, 2, closed='right'): 135,
 Interval(2, 3, closed='right'): 100,
 Interval(3, 5, closed='right'): 118,
 Interval(5, 10, closed='right'): 128,
 Interval(10, 100, closed='right'): 137,
 Interval(100, 1000, closed='right'): 0,
 'avg_hours': 320.8698402777778,
 'avg_days': 13.369576678240742,
 'std_hours': 57.45390885222046,
 'std_days': 2.3939128688425195}

**Questionnaire of most frequent user has been filled out ~ each 14 days, i.e. one period ~ 14 days**

In [12]:
# read in codebook and reduce to columns and rows of interest
cb = pd.read_excel('../../data/d00_helpers/codebook/ch/rki_heart.xlsx', sheet_name='FollowUp', header=4)
cb = cb[cb.elementtype=='question']
cb = cb.iloc[:, :30]